In [1]:
import hepfile as hf
import awkward as ak
import numpy as np

In [2]:
# test dictionary
d = [
    {
    'jet': {
        'px': [1,2,3],
        'py': [1,2,3]
     },
    'muons': {
        'px': [1,2,3],
        'py': [1,2,3]
     },
    'other': 'this'
    },
    {
    'jet': {
        'px': [3,4,6,7],
        'py': [3,4,6,7]
     },
    'muons': {
        'px': [3,4,6,7],
        'py': [3,4,6,7],
        },
    'other': 'this'
    }
]

In [3]:
awk = ak.Array(d)

In [4]:
out = 'test.hdf5'
ak_dict = hf.dict_tools.dictlike_to_hepfile(d, out)
ak_dict

jet px int64
jet py int64
muons px int64
muons py int64
other <class 'str'>
Writing the hdf5 file from the awkward array...
Metadata added


<Array [{jet: {...}, muons: {...}, ...}, ...] type='2 * {jet: {px: var * in...'>

In [5]:
new_dict = {'jet': {'px': [10, 100], 'py': [0, 0]},
            'muons': {'px': [5, 1000], 'py': [0, -1]},
            'other': 2
            }

mod = hf.dict_tools.append(ak_dict, new_dict)

In [6]:
for group in mod.fields:
    print(group)
    if len(mod[group].fields) == 0:
        print('singleton')
        continue
    for dataset in mod[group].fields:
        print(mod[group][dataset])

jet
[[1, 2, 3], [3, 4, 6, 7], [10, 100]]
[[1, 2, 3], [3, 4, 6, 7], [0, 0]]
muons
[[1, 2, 3], [3, 4, 6, 7], [5, 1000]]
[[1, 2, 3], [3, 4, 6, 7], [0, -1]]
other
singleton


In [7]:
help(hf.dict_tools.append)

Help on function append in module hepfile.dict_tools:

append(ak_dict: 'ak.Record', new_dict: 'dict') -> 'ak.Record'
    Append a new event to an existing awkward dictionary with events
    
    Args:
        ak_dict (ak.Record): awkward Record of data
        new_dict (dict): Dictionary of value to append to ak_dict. All keys must match ak_dict!
    Return:
        Awkward Record of awkward arrays with the new_dict appended



# test with pandas

In [8]:
import pandas as pd

In [9]:
d = { 'event1' : 
    {
    'jet': {
        'px': [1,2,3],
        'py': [1,2,3]
     },
    'muons': {
        'px': [1,2,3],
        'py': [1,2,3]
     },
    'other': 'this'
    },
     'event2' : 
    {
    'jet': {
        'px': [3,4,6,7],
        'py': [3,4,6,7]
     },
    'muons': {
        'px': [3,4,6,7],
        'py': [3,4,6,7],
        },
    'other': 'this'
    }
    }

df = pd.DataFrame(d)

In [10]:
awk = hf.dict_tools.dictlike_to_hepfile(df, 'dataframe-to-hepfile.h5', how_to_pack='classic', write_hepfile=False)

Metadata added


In [12]:
data, bucket = hf.load('dataframe-to-hepfile.h5')

Building the indices...

Built the indices!
Data is read in and input file is closed.


In [20]:
awk

{'_GROUPS_': {'_SINGLETONS_GROUP_': ['COUNTER', 'other'],
  'jet': ['njet', 'px', 'py'],
  'muons': ['nmuons', 'px', 'py']},
 '_MAP_DATASETS_TO_COUNTERS_': {'_SINGLETONS_GROUP_': '_SINGLETONS_GROUP_/COUNTER',
  'jet': 'jet/njet',
  'jet/px': 'jet/njet',
  'jet/py': 'jet/njet',
  'muons': 'muons/nmuons',
  'muons/px': 'muons/nmuons',
  'muons/py': 'muons/nmuons',
  'other': '_SINGLETONS_GROUP_/COUNTER'},
 '_LIST_OF_COUNTERS_': ['_SINGLETONS_GROUP_/COUNTER',
  'jet/njet',
  'muons/nmuons'],
 '_SINGLETONS_GROUP_/COUNTER': [1, 1],
 '_MAP_DATASETS_TO_DATA_TYPES_': {'_SINGLETONS_GROUP_/COUNTER': int,
  'jet/njet': int,
  'jet/px': int,
  'jet/py': int,
  'muons/nmuons': int,
  'muons/px': int,
  'muons/py': int,
  'other': str},
 '_META_': {},
 'jet/njet': [3, 4],
 'jet/px': [1, 2, 3, 3, 4, 6, 7],
 'jet/py': [1, 2, 3, 3, 4, 6, 7],
 'muons/nmuons': [3, 4],
 'muons/px': [1, 2, 3, 3, 4, 6, 7],
 'muons/py': [1, 2, 3, 3, 4, 6, 7],
 'other': ['this', 'this']}

In [22]:
data.keys() - awk.keys()

{'_LIST_OF_DATASETS_',
 '_MAP_DATASETS_TO_INDEX_',
 '_NUMBER_OF_BUCKETS_',
 '_PROTECTED_NAMES_',
 '_SINGLETONS_GROUP_',
 '_SINGLETONS_GROUP_/COUNTER_INDEX',
 'jet/njet_INDEX',
 'muons/nmuons_INDEX'}